In [13]:
import requests

res = requests.get("https://www.sigma-computer.com/search?search=laptop&submit_search=&route=product%2Fsearch")
res

<Response [200]>

In [17]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

base_url = "https://www.sigma-computer.com/search?search=laptop&submit_search=&route=product%2Fsearch"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

def get_soup(url):
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return BeautifulSoup(response.content, 'html.parser')
    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        return None

def scrape_page(soup):
    products = []
    for item in soup.find_all('div', class_='product-layout'):
        name = item.find('h4').get_text(strip=True) if item.find('h4') else 'N/A'
        price = item.find('p', class_='price').get_text(strip=True) if item.find('p', class_='price') else 'N/A'
        availability = item.find('span', class_='stock').get_text(strip=True) if item.find('span', class_='stock') else 'Not Available'
        image_url = item.find('img')['src'] if item.find('img') else 'N/A'
        link = item.find('a')['href'] if item.find('a') else 'N/A'
        products.append({
            'name': name,
            'price': price,
            'availability': availability,
            'image_url': image_url,
            'link': link
        })
    return products

def get_next_page_url(soup):
    next_page = soup.find('a', class_='next')
    if next_page:
        return next_page['href']
    return None

def main():
    all_products = []
    url = base_url

    while url:
        soup = get_soup(url)
        if soup:
            products = scrape_page(soup)
            all_products.extend(products)
            url = get_next_page_url(soup)
            if url:
                url = base_url.rsplit('?', 1)[0] + url
                # Add a delay to avoid overwhelming the server
                time.sleep(2)

    # Save to CSV
    df = pd.DataFrame(all_products)
    df.to_csv('Sigma.csv', index=False)
    print("Data saved to Sigma.csv")

    # Read the CSV file
    df = pd.read_csv('Sigma.csv')
    print(df)

if __name__ == "__main__":
    main()


Data saved to Sigma.csv
                                                  name          price  \
0    Laptop Dell Vostro 3520 - Intel Core i5 1235U ...  19199 LE20000   
1    HP Victus16-r1062ne Intel Core i5 14450HX-RTX ...  43999 LE45000   
2    HP Victus16-r1066ne Intel Core i5 14450HX-RTX ...  43499 LE45000   
3    Sigma laptop Backpack(black) And LAPTOP STAND ...      449 LE650   
4    MSI Modern 15 F13MG Laptop - Intel Core i5-133...  21749 LE24500   
..                                                 ...            ...   
735  G3 15-3579 Gaming Laptop - Intel Core I7 - 8GB...  15000 LE15900   
736  G3 15-3579 Gaming Laptop - Intel Core i7 - 16G...  16000 LE20899   
737  RAPOO 3510 Plus 2.4GHz Wireless Optical Fabric...      275 LE500   
738  Western Digital 1TB Hard Disk Drive - 5400 RPM...    2399 LE3999   
739  Lenovo Ideapad 320 Laptop - Intel Core i5-8250...    8900 LE8900   

      availability                         image_url  \
0         In Stock    image/products/173588